# CO542 - Neural Networks and Fuzzy Systems
## E/19/129 - K.H. Gunawardana

### Lab 03: Generalized Hebbian Learning (GHL)

#### Task 01: GHL for 2D Synthetic Data

In [1]:
import torch

ModuleNotFoundError: No module named 'tourch'